In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Dropout, Flatten, Activation, Lambda, Input
from keras.layers.convolutional import ZeroPadding2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers.core import Lambda
from keras import initializers
from keras.utils import get_file
import tensorflow as tf
import numpy as np
import os

Using TensorFlow backend.


In [2]:
import collections

path = '/work/04381/ymarathe/maverick/yearbook/keras_yearbook/'
data_path = '/work/04381/ymarathe/maverick/yearbook/'

f = open(data_path + 'yearbook_train.txt', 'r')

freq = {};
normal_const = 0;

for line in f:
    line = line.rstrip()
    image, year = line.split("\t")
    if year in freq:
        freq[year] += 1
    else:
        freq[year] = 1

normal_const = np.sum(freq.values())
for key in freq:
    freq[key] = freq[key]/float(normal_const);
    
sorted_freq = collections.OrderedDict(sorted(freq.items()))

idx = 0;
class_weights_train = {}
idx2year = {}

for key in sorted_freq:
    class_weights_train[idx] = sorted_freq[key]
    idx2year[idx] = key
    idx += 1

In [3]:
f = open(data_path + 'yearbook_train.txt', 'r')

fnameToGenderTr = {}

for line in f:
    line = line.rstrip()
    image, year = line.split("\t")
    gender, imname = image.split("/")
    if gender is 'M':
        encodeGender = 1
    elif gender is 'F':
        encodeGender = 0
    fnameToGenderTr[image] = encodeGender

f = open(data_path + 'yearbook_valid.txt', 'r')

fnameToGenderVd = {}

for line in f:
    line = line.rstrip()
    image, year = line.split("\t")
    gender, imname = image.split("/")
    if gender is 'M':
        encodeGender = 1
    elif gender is 'F':
        encodeGender = 0
    fnameToGenderVd[image] = encodeGender

In [4]:
def gen_batches(path, gen = ImageDataGenerator(), shuffle=True, class_mode="categorical", batch_size=32, 
                target_size=(171, 186)):
    return gen.flow_from_directory(path, shuffle=shuffle, batch_size=batch_size, target_size=target_size, 
                                   class_mode=class_mode)

def gen_batches_flow(path, gen = ImageDataGenerator(), shuffle=True, batch_size=32):
    return gen.flow(path, shuffle=shuffle, batch_size=batch_size)

In [5]:
def vgg19_conv_layers_sequential():
    model = Sequential()
    
    model.add(ZeroPadding2D((1, 1), input_shape=(171, 186, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))

    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    vgg_pretrain_weights = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    "https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                    cache_subdir='models')
    
    model.load_weights(vgg_pretrain_weights)
    
    return model

def vgg19_full_sequential():
    model = Sequential()
    
    model.add(ZeroPadding2D((1, 1), input_shape=(224, 224, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))

    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))
    
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(1000, activation='softmax'))
    
    vgg_pretrain_weights = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                                                    'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                                                    cache_subdir='models',
                                                    file_hash='cbe5617147190e668d6c5d5026f83318')
    
    model.load_weights(vgg_pretrain_weights)
    
    return model

In [6]:
def vgg19_conv_functional(img_input):
    
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    return x

In [7]:
import keras.backend as K
from keras.preprocessing.image import load_img, img_to_array

class PseudoLabeling:
    def __init__(self, model, train, valid, path = None):
        self.model = model
        self.train = train
        self.valid = valid
        self.path = path
        self.val_pred_labels = None
        
    def predict_labels(self, batch):
        pred_probs = self.model.predict_generator(batch, steps = ((batch.samples/batch.batch_size) + 1))
        pred_labels = np.zeros_like(pred_probs)
        pred_labels[np.arange(len(pred_labels)), pred_probs.argmax(1)] = 1
        return pred_labels
    
    def predict_validation_labels(self):
        self.val_pred_labels = self.predict_labels(self.valid)
        
    def reset_validation_labels(self):
        self.val_pred_labels = None
        
    def get_validation_labels(self, idx_array = None):
        if self.val_pred_labels is None:
            raise Exception('Validation labels not populated. \
                            Call predict_validation_labels on Pseudolabeling object \
                            to populate validation labels first')
        validation_labels = []
        if idx_array is None:
            return self.val_pred_labels
        else:
            for idx in idx_array:
                validation_labels.append(self.val_pred_labels[idx])
                
            return validation_labels   

    def next(self, *args, **kwargs):
        batch_train_x, batch_train_y = next(self.train)
        
        batch_valid_idx_array, batch_valid_cur_idx, batch_size = next(self.valid.index_generator)
        batch_valid_x = np.zeros((batch_size,) + self.valid.image_shape, dtype=K.floatx())
        grayscale = self.valid.color_mode == 'grayscale'
        
        for i, j in enumerate(batch_valid_idx_array):
            fname = self.valid.filenames[j]
            img = load_img(
                  os.path.join(self.valid.directory, fname),
                  grayscale=grayscale,
                  target_size=self.valid.target_size)
            x = img_to_array(img, data_format=self.valid.data_format)
            x = self.valid.image_data_generator.random_transform(x)
            x = self.valid.image_data_generator.standardize(x)
            batch_valid_x[i] = x
        
        batch_valid_y = self.get_validation_labels(batch_valid_idx_array);
        
        n0 = np.concatenate([batch_train_x, batch_valid_x])
        n1 = np.concatenate([batch_train_y, batch_valid_y])
        return (n0, n1)

In [8]:
import math
import os
import numpy as np
from keras.preprocessing.image import Iterator
from keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing.image import apply_transform, transform_matrix_offset_center
import keras.backend as K
from skimage import exposure
data_path = '/work/04381/ymarathe/maverick/yearbook/'

class RegressDataGen:
    def __init__(self, directory, map_file, target_size = (171, 186, 3), 
                 class_weights_train = None, multi_output=False, do_augmentation=True, 
                 samplewise_center = True,
                 samplewise_std_deviation = True):
        self.directory = directory
        self.map_file = map_file
        self.filenames = []
        self.map = {}
        self.fnameToGender = {}
        self.target_size = target_size
        self.populate_filenames()
        self.populate_mapping()
        self.regressIter = None
        self.steps = 0
        self.samplewise_center = samplewise_center
        self.samplewise_std_deviation = samplewise_std_deviation
        self.height_shift_range = 0.2
        self.width_shift_range = 0.2
        self.zoom_range = (0.5, 0.5)
        self.do_augmentation = do_augmentation
        self.class_weights_train = class_weights_train
        self.equalizehist = False
        self.multi_output = multi_output
        
    def _recursive_list(self, subpath):
        return sorted(
            os.walk(subpath, followlinks=False), key=lambda tpl: tpl[0])
    
    def populate_mapping(self):
        f = open(self.map_file, 'r')

        for line in f:
            line = line.rstrip()
            image, year = line.split("\t")
            gender, imfilename = image.split("/")
            if gender is 'M':
                encodeGender = 1
            elif gender is 'F':
                encodeGender = 0
            self.fnameToGender[image] = encodeGender
            self.map[image] = year
            
    def populate_filenames(self):
        base_dir = self.directory
        for root, _, files in self._recursive_list(base_dir):
            for fname in files:
                if fname.lower().endswith('.' + 'png'):
                    self.filenames.append(os.path.relpath(os.path.join(root, fname), base_dir))
                    
    def preprocess(self, x):
        if self.equalizehist:
            x = exposure.equalize_hist(x)
            
        return x
            
    def augment_data(self, x):
        
        if self.height_shift_range:
            tx = np.random.uniform(-self.height_shift_range, self.height_shift_range) * x.shape[0]
        else:
            tx = 0
        
        if self.width_shift_range:
            ty = np.random.uniform(-self.width_shift_range, self.width_shift_range) * x.shape[1]
        else:
            ty = 0
        
        if tx != 0 or ty != 0:
            shift_matrix = np.array([[1, 0, tx], [0, 1, ty], [0, 0, 1]])
            transform_matrix = shift_matrix
            
        zx, zy = np.random.uniform(self.zoom_range[0], self.zoom_range[1], 2)
        
        if zx != 1 or zy != 1:
            zoom_matrix = np.array([[zx, 0, 0], [0, zy, 0], [0, 0, 1]])
            transform_matrix = zoom_matrix if transform_matrix is None else np.dot(transform_matrix, zoom_matrix) 
        
        if transform_matrix is not None:
            h = self.target_size[0]
            w = self.target_size[1]
            img_channel_axis = 2
            
            transform_matrix = transform_matrix_offset_center(transform_matrix, h, w)
            
            x = apply_transform(
                x,
                transform_matrix,
                img_channel_axis,
                fill_mode='nearest',
                cval=0)
            
        return x
            
    def flow_from_directory(self, batch_size = 32, shuffle = True, seed = 42):
        
        self.regressIter = Iterator(len(self.filenames), batch_size = batch_size, shuffle = shuffle, seed = seed)
        
        if self.do_augmentation:
            factor = 2
        else:
            factor = 1
        
        self.steps = math.ceil(len(self.filenames)/batch_size) * factor
        
        return self
    
    def next(self, *args, **kwargs):
           
        idx_array, cur_idx, bs = next(self.regressIter.index_generator)
        
        batch_x = np.zeros(tuple([len(idx_array)] + list(self.target_size)), dtype=K.floatx())
        
        if self.multi_output:
            batch_y = np.zeros(tuple([len(idx_array)]), dtype=K.floatx())
            batch_y_gender = np.zeros(tuple([len(idx_array)]), dtype=K.floatx())
            batch_y = np.stack((batch_y, batch_y_gender), axis = -1)
        else:
            batch_y = np.zeros(tuple([len(idx_array)]), dtype=K.floatx())
        
        if self.class_weights_train is not None:
            sample_weights = np.ones(tuple([len(idx_array)]), dtype=K.floatx())
        
        for i, j in enumerate(idx_array):
            fname = self.filenames[j]
            print fname
            img = load_img(
                  os.path.join(self.directory, fname),
                  grayscale = True,
                  target_size= self.target_size)
            x = np.array(img_to_array(img, data_format='channels_last'))
            x = self.preprocess(x)
            batch_x[i] = x
            if self.multi_output:
                batch_y[i, :] = self.map[fname], self.fnameToGender[fname]
            else:
                batch_y[i] = self.map[fname]
            
            if self.class_weights_train is not None:
                if self.multi_output:
                    sample_weights[i] = self.class_weights_train[batch_y[i, 0].astype('int').astype('str')]
                else:
                    sample_weights[i] = self.class_weights_train[batch_y[i].astype('int').astype('str')]
        
        if self.samplewise_center:
            for x in batch_x:
                x -= np.mean(x)
        
        if self.samplewise_std_deviation:
            for x in batch_x:
                x /= np.std(x)
        
        if self.do_augmentation:
            for x in batch_x:
                x = self.augment_data(x)
                
        if self.class_weights_train is not None:
            return (batch_x, batch_y, sample_weights)
        else:
            return (batch_x, batch_y)

train = RegressDataGen(data_path + 'train',
                             data_path + 'yearbook_train.txt', class_weights_train = sorted_freq)
valid = RegressDataGen(data_path + 'valid',
                             data_path + 'yearbook_valid.txt', class_weights_train = sorted_freq)

train = train.flow_from_directory()
valid = valid.flow_from_directory()

In [9]:
# Rounding layer
from keras.layers import Layer
class Round(Layer):
    def __init__(self, **kwargs):
        super(Round, self).__init__(**kwargs)

    def get_output(self, train=False):
        X = self.get_input(train)
        return K.round(X)

    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(Round, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.layers import Input
from keras.models import Model
from keras.layers import merge

mean_value = 0
for key in freq:
    mean_value += freq[key] * float(key)

img_input = Input(shape=(171, 186, 3))
x = vgg19_conv_functional(img_input)
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(4096, kernel_initializer='glorot_normal', bias_initializer=keras.initializers.Ones())(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x_year = Dense(1, bias_initializer=keras.initializers.Constant(mean_value))(x)
out_year = Round()(x_year)
x_class = Dense(1, bias_initializer=keras.initializers.Constant(0.5))(x)
out_class = Round()(x_class)
out = merge((out_year, out_class), 'concat')

model = Model([img_input], [out])

lr = 1e-2

model.compile(Adam(lr=lr), loss=['mse'], 
              metrics=['mae'], 
            )

def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

train = RegressDataGen(data_path + 'train',
                       data_path + 'yearbook_train.txt', 
                       class_weights_train = sorted_freq, 
                       multi_output=True,
                       do_augmentation=True
                       )
valid = RegressDataGen(data_path + 'valid',
                       data_path + 'yearbook_valid.txt',
                       class_weights_train = sorted_freq,
                       do_augmentation=False,
                       multi_output=True
                      )

train = train.flow_from_directory()
valid = valid.flow_from_directory()

In [10]:
with tf.device('/gpu:0'):
    
    model.fit_generator(train, steps_per_epoch = train.steps, epochs = 1,                                
                               validation_data = valid, 
                               validation_steps = valid.steps,
                               class_weight=class_weights_train,
                               callbacks=[LearningRateScheduler(lr_schedule),
                               ModelCheckpoint(path + 'exp28.h5', save_best_only=True)]
                       )

NameError: name 'model' is not defined

In [11]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.layers import Input
from keras.models import Model

train = RegressDataGen(data_path + 'train',
                       data_path + 'yearbook_train.txt', 
                       class_weights_train = sorted_freq,
                       do_augmentation = True)
valid = RegressDataGen(data_path + 'valid',
                       data_path + 'yearbook_valid.txt',
                       class_weights_train = sorted_freq, 
                       do_augmentation = False)

train = train.flow_from_directory()
valid = valid.flow_from_directory()

mean_value = 0
for key in freq:
    mean_value += freq[key] * float(key)
    
model = vgg19_conv_layers_sequential()
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(4096, kernel_initializer='glorot_normal', bias_initializer=keras.initializers.Ones()))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1, bias_initializer = keras.initializers.Constant(mean_value)))
model.add(Round())

for layer in model.layers:
    if type(layer) != Dense:
        layer.trainable = False

lr = 1e-2

model.compile(Adam(lr=lr), loss='mse', metrics=['accuracy', 'mae'])

def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

model.load_weights(path + 'exp27.h5')

In [12]:
batch_x, batch_y, sample_weight = next(valid)
y_pred = model.predict(batch_x)
for i, j in enumerate(y_pred):
    print j, batch_y[i], abs(j - batch_y[i])

F/001510.png
M/000603.png
M/001875.png
F/002578.png
M/001851.png
M/001562.png
F/000176.png
F/002276.png
M/001531.png
F/000397.png
M/000399.png
M/001685.png
F/002640.png
M/001101.png
F/000292.png
F/000089.png
F/000600.png
F/001944.png
M/001121.png
F/000438.png
F/002722.png
M/000944.png
F/001951.png
F/000138.png
M/000043.png
F/000997.png
F/002092.png
F/001882.png
F/001028.png
M/001014.png
M/001349.png
F/001746.png
[ 1942.72814941] 1936.0 [ 6.72814941]
[ 1963.87133789] 1955.0 [ 8.87133789]
[ 1986.94189453] 1984.0 [ 2.94189453]
[ 1989.73266602] 2005.0 [ 15.26733398]
[ 1987.32287598] 1981.0 [ 6.32287598]
[ 1981.08483887] 1977.0 [ 4.08483887]
[ 1984.91040039] 1984.0 [ 0.91040039]
[ 1991.95654297] 1984.0 [ 7.95654297]
[ 1956.23022461] 1955.0 [ 1.23022461]
[ 1990.31982422] 1992.0 [ 1.68017578]
[ 1959.25756836] 1955.0 [ 4.25756836]
[ 1953.85705566] 1947.0 [ 6.85705566]
[ 1982.25598145] 1979.0 [ 3.25598145]
[ 1940.3260498] 1947.0 [ 6.6739502]
[ 1935.20947266] 1936.0 [ 0.79052734]
[ 1950.09033203

In [15]:
import matplotlib.pyplot as plt

In [22]:
fname = 'F/000176.png'
img = load_img(
            os.path.join(data_path + 'valid', fname),
            grayscale = True,
            target_size = (171, 186, 3))
x = np.array(img_to_array(img, data_format='channels_last'))
np.shape(x)

(171, 186, 1)